In [1]:
%pip install langgraph==0.2.74
%pip install langgraph-checkpoint==2.0.16
%pip install langgraph-sdk==0.1.53
%pip install langsmith--0.3.8
%pip install lazy_loader==0.4

   ---------------------------------------- 0.0/151.4 kB ? eta -:--:--
   ---------------- ----------------------- 61.4/151.4 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 151.4/151.4 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/45.8 kB ? eta -:--:--
   ---------------------------------------- 45.8/45.8 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/50.3 kB ? eta -:--:--
   ---------------------------------------- 50.3/50.3 kB 1.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/117.5 kB ? eta -:--:--
   ---------------------------------------- 117.5/117.5 kB 3.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/72.7 kB ? eta -:--:--
   ---------------- ----------------------- 30.7/72.7 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------  71.7/72.7 kB 653.6 kB/s eta 0:00:01
   ---------------------------------------- 72.7/72.7 kB 573.1 kB/s eta 0:00:00
  Attempting uninstall: langgraph-checkpoint
    Found existing installation: langgraph-checkpoint 2.1.2
    Uninstalling langgraph-checkpoint-2.1.2:
      Successfully uninstalled langgraph-checkpoint-2.1.2
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/45.4 kB ? eta -:--:--
   --------- ------------------------------ 10.2/45.4 kB ? eta -:--:--
   ------------------ --------------------- 20.5/45.4 kB 330.3 kB/s eta 0:00:01
   ---------------------------------------- 45.4/45.4 kB 374.5 kB/s eta 0:00:00
  Attempting uninstall: langgraph-sdk
    Found existing installation: langgraph-sdk 0.1.74
    Uninstalling langgraph-sdk-0.1.74:
      Successfully uninstalled langgraph-sdk-0.1.74
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement langsmith--0.3.8 (from versions: none)
ERROR: No matching distribution found for langsmith--0.3.8

[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain_core.caches import BaseCache
from langchain_core.callbacks.base import Callbacks

#.env파일 로드
load_dotenv()

ChatOpenAI.model_rebuild()
model = ChatOpenAI(model="gpt-4o-mini")
model.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_c4585b5b9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-cd9c9048-45d0-4699-9e52-6e228a556086-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
#상태 정의하기
#랭그래프에서 상태는 언어 모델이 임무를 수행하면서 현재 상태를 명확히 관리 할 수 있도록 돕는 요소
#랭그래프는 여러 노드(여러 AI 에이전트)가  각자 맡은 일을 수행하도록 구성되고, 이 노드들은 상황에 맞게 작업할 수 있도록 필요한 정보를 주고 받아야 함.
#그래서 state(상태) 클래스에 필요한 데이터 형식을 최대한 명확하게 정의하고, 각 노드에서 처리된 결과를 state에 저장하여 다음 작업을 수행할 노드로 전달함.
#랭그래프에서는 각 작업을 진행하는 데 필요한 항목들의 자료 형태를 상태에 미리 정해 놓아야 각 노드가 필요한 정보를 정확히 받아 작업이 순차로 잘 이어짐짐

In [3]:
from typing import Annotated #annotated: 타입 힌트
from typing_extensions import TypedDict #TypedDict: 딕셔너리 타입을 정의함

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

class State(TypedDict):
    """
    State 클래스는 Tydict를 상속받음
    
    속성:
        messages (Annotated[list[str], add_messages]): 메시지들은 "list"타입을 가집니다.
        'add_messages'함수는 이 상태 키가 어떻게 업데이트 되어야 하는지를 정의합니다.
        (이 경우, 메시지를 덮어쓰는 대신 리스트에 추가합니다.)
    """

    messages: Annotated[list[str], add_messages]

#StateGraph 클래스를 사용하여 State 타입의 그래프 생성
graph_builder = StateGraph(State)

In [ ]:
#노드 생성하기

#랭그래프는 각 노드에서 처리한 결과를 상태에서 관리하고, 각 노드를 엣지로 연결하는 그래프 형태로 표현하여 대화나 데이터 흐름을 관리함
#노드는 그래프의 한 지점을 의미하며, 하나의 단계를 표현함
#사용자가 질문하면 답변을 생성하는 generate라는 노드를 가진 랭그래프를 만듦
#generate 노드: 기존 대화 내용에 기반하여 GPT로 답변을 생성하는 역할을 함함

In [4]:
def generate(state: State):
    f"""
    주어진 상태를 기반으로 챗봇의 응답 메시지를 생성함.

    매개변수:
    state(State): 현재 대화 상태를 나타내는 객체로, 이전 메시지들이 포함되어 있음.

    반환값:
    dict: 모델이 생성한 응답 메시지를 포함하는 딕셔너리
        형식은 {"messages":[응답 메시지]}입니다.
    """

    return {"messages":[model.invoke(state["messages"])]}

graph_builder.add_node("generate", generate)

In [ ]:
#엣지 설정하기

#START, END 노드를 앞뒤로 지정하고, 엣지를 설정함
#앞서 만든 graph_builder에 연결 관계를 선언하여 흐름을 정의함

graph_builder.add_edge(START, "generate")
graph_builder.add_edge("generate", END)

graph = graph_builder.compile()
